In [1]:
import pandas as pd
import numpy as np

# Import Data

In [2]:
FF = pd.read_csv('F-F_Research_Data_5_Factors_2x3.csv', header = 2, nrows= 687)

In [3]:
FF.rename(columns={FF.columns[0]: "Date" }, inplace = True)
FF.set_index('Date', inplace = True)

In [4]:
Momentum = pd.read_csv('F-F_Momentum_Factor.csv', header = 11, nrows= 1125)
Momentum.rename(columns={Momentum.columns[0]: "Date",
                        Momentum.columns[1]: "UMD"}, inplace = True)
Momentum.set_index('Date', inplace = True)
Momentum

,UMD
Date,
192701,0.44
192702,-1.32
192703,3.59
192704,4.17
192705,2.96
...,...
202005,0.67
202006,-0.76
202007,7.61


In [5]:
FF_mom = FF.merge(Momentum, how='left', on='Date')

In [6]:
FF_mom

,Mkt-RF,SMB,HML,RMW,CMA,RF,UMD
Date,,,,,,,
196307,-0.39,-0.47,-0.83,0.66,-1.15,0.27,1.00
196308,5.07,-0.79,1.67,0.40,-0.40,0.25,1.03
196309,-1.57,-0.48,0.18,-0.76,0.24,0.27,0.16
196310,2.53,-1.29,-0.10,2.75,-2.24,0.29,3.14
196311,-0.85,-0.84,1.71,-0.45,2.22,0.27,-0.75
...,...,...,...,...,...,...,...
202005,5.58,1.92,-4.95,0.71,-3.28,0.01,0.67
202006,2.46,1.94,-2.22,0.04,0.34,0.01,-0.76
202007,5.77,-3.03,-1.31,0.55,1.06,0.01,7.61


In [7]:
import statsmodels.api as sm
from scipy import stats
from sklearn.linear_model import LinearRegression

In [8]:
FF_mom[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']]

,Mkt-RF,SMB,HML,RMW,CMA
Date,,,,,
196307,-0.39,-0.47,-0.83,0.66,-1.15
196308,5.07,-0.79,1.67,0.40,-0.40
196309,-1.57,-0.48,0.18,-0.76,0.24
196310,2.53,-1.29,-0.10,2.75,-2.24
196311,-0.85,-0.84,1.71,-0.45,2.22
...,...,...,...,...,...
202005,5.58,1.92,-4.95,0.71,-3.28
202006,2.46,1.94,-2.22,0.04,0.34
202007,5.77,-3.03,-1.31,0.55,1.06


# 1.3.1 Replicate tables

## Table 1

In [11]:
table[table.index<201300]

,Mkt-RF,SMB,HML,RMW,CMA
Date,,,,,
196307,-0.39,-0.47,-0.83,0.66,-1.15
196308,5.07,-0.79,1.67,0.40,-0.40
196309,-1.57,-0.48,0.18,-0.76,0.24
196310,2.53,-1.29,-0.10,2.75,-2.24
196311,-0.85,-0.84,1.71,-0.45,2.22
...,...,...,...,...,...
201208,2.55,0.43,1.21,-1.42,-0.72
201209,2.73,0.65,1.53,-1.32,1.60
201210,-1.76,-0.82,3.68,-1.50,2.27


In [10]:
table = FF_mom[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']]

print_model = pd.DataFrame({"empty": range(0)})

colnames = table.columns

for col in table.columns:
    y = table[col]
    X = table.drop(col, axis = 1)
    X = sm.add_constant(X)
    X.rename(columns={'const': "Intercept Annual" }, inplace = True)
    
    model = sm.OLS(y, X).fit()
    
    t = pd.DataFrame(model.tvalues)
    
    params = pd.DataFrame(model.params)
    params.iloc[0] = params.iloc[0]*12/100 # Consistent with AQR, I made the intercept as per annum
    params.iloc[0] =  params.iloc[0].apply("{:.2%}".format)
    
    r2 = pd.Series(model.rsquared).to_frame(0)
    r2.iloc[0] = r2.iloc[0].apply("{:.2%}".format)
    
    params = params.append(r2)
    
    out_put = params.merge(t, how='left', left_index=True, right_index=True)
    
    out_put.rename(columns={out_put.columns[0]: col,
                            out_put.columns[1]: ""},
                   index={out_put.index[-1]: 'R-Squared'}, inplace=True)
    
    print_model = pd.concat([print_model, out_put], join="outer", axis = 1)

print_model = print_model.iloc[:,1:]
order = colnames.insert(0,'Intercept Annual').insert(len(colnames)+1, 'R-Squared')
print_model.reindex(order).replace(np.nan, "").T

,Intercept Annual,Mkt-RF,SMB,HML,RMW,CMA,R-Squared
Mkt-RF,9.33%,,0.286702,0.126619,-0.311583,-0.933833,22.57%
,5.06232,,5.35878,1.74226,-4.17582,-8.87729,
SMB,3.03%,0.140931,,0.109783,-0.425204,-0.157677,16.91%
,2.30929,5.35878,,2.15711,-8.43411,-2.03028,
HML,-1.09%,0.0349957,0.0617267,,0.174267,1.0312,48.60%
,-1.10146,1.74226,2.15711,,4.45,23.9583,
RMW,4.57%,-0.080013,-0.222131,0.161916,,-0.2886,16.07%
,4.89228,-4.17582,-8.43411,4.45,,-5.22757,
CMA,2.98%,-0.110922,-0.0381014,0.443178,-0.133493,,54.09%
,4.6832,-8.87729,-2.03028,23.9583,-5.22757,,


## Table 2

In [12]:
table = FF_mom[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'UMD']]

print_model = pd.DataFrame({"empty": range(0)})

colnames = table.columns

for col in table.columns:
    y = table[col]
    X = table.drop(col, axis = 1)
    X = sm.add_constant(X)
    X.rename(columns={'const': "Intercept Annual" }, inplace = True)
    
    model = sm.OLS(y, X).fit()
    params = pd.DataFrame(model.params)
    
    t = pd.DataFrame(model.tvalues)
    
    params.iloc[0] = params.iloc[0]*12/100
    params.iloc[0] =  params.iloc[0].apply("{:.2%}".format)
    
    r2 = pd.Series(model.rsquared).to_frame(0)
    r2.iloc[0] = r2.iloc[0].apply("{:.2%}".format)
    
    params = params.append(r2)
    
    out_put = params.merge(t, how='left', left_index=True, right_index=True)
    
    out_put.rename(columns={out_put.columns[0]: col,
                            out_put.columns[1]: ""},
                   index={out_put.index[-1]: 'R-Squared'}, inplace=True)
    
    print_model = pd.concat([print_model, out_put], join="outer", axis = 1)

print_model = print_model.iloc[:,1:]
order = colnames.insert(0,'Intercept Annual').insert(len(colnames)+1, 'R-Squared')
print_model.reindex(order).replace(np.nan, "").T

,Intercept Annual,Mkt-RF,SMB,HML,RMW,CMA,UMD,R-Squared
Mkt-RF,10.33%,,0.287056,0.0541088,-0.275803,-0.868452,-0.137651,24.11%
,5.59959,,5.41582,0.725486,-3.69995,-8.21765,-3.72556,
SMB,2.84%,0.143847,,0.120357,-0.429318,-0.16466,0.0210006,16.99%
,2.13473,5.41582,,2.2875,-8.46897,-2.1062,0.795236,
HML,0.09%,0.0142727,0.0633546,,0.190311,1.0077,-0.127213,51.92%
,0.0879342,0.725486,2.2875,,5.01168,24.1089,-6.86212,
RMW,4.07%,-0.0714499,-0.221946,0.186907,,-0.303242,0.0527652,17.02%
,4.28873,-3.69995,-8.46897,5.01168,,-5.49517,2.79351,
CMA,2.61%,-0.103882,-0.0393049,0.456966,-0.140018,,0.0385193,54.69%
,4.04562,-8.21765,-2.1062,24.1089,-5.49517,,3.00378,


## Table 3

In [13]:
dev = pd.read_excel('The Devil in HMLs Details Factors Monthly.xlsx', header = 18)
dev = dev[['DATE','USA']]
dev['Month'] = dev.DATE.str[:2]
dev['Year'] = dev.DATE.str[-4:]
dev['DATE'] = dev.Year+dev.Month
dev.rename(columns={dev.columns[0]: "Date",
                        dev.columns[1]: "DEV"}, inplace = True)
dev=dev[['Date', 'DEV']]
dev.Date=dev['Date'].astype(str).astype(int)
dev.set_index('Date', inplace = True)

In [14]:
dev.index

Int64Index([192607, 192608, 192609, 192610, 192611, 192612, 192701, 192702,
            192703, 192704,
            ...
            202001, 202002, 202003, 202004, 202005, 202006, 202007, 202008,
            202009, 202010],
           dtype='int64', name='Date', length=1132)

In [15]:
FF_mom_DEV = FF_mom.merge(dev, how='left', on="Date")
FF_mom_DEV['HML-DEV'] = FF_mom_DEV.HML-FF_mom_DEV.DEV
FF_mom_DEV.head()

,Mkt-RF,SMB,HML,RMW,CMA,RF,UMD,DEV,HML-DEV
Date,,,,,,,,,
196307,-0.39,-0.47,-0.83,0.66,-1.15,0.27,1.00,-0.005923,-0.824077
196308,5.07,-0.79,1.67,0.40,-0.40,0.25,1.03,0.002805,1.667195
196309,-1.57,-0.48,0.18,-0.76,0.24,0.27,0.16,0.002653,0.177347
196310,2.53,-1.29,-0.10,2.75,-2.24,0.29,3.14,-0.021129,-0.078871
196311,-0.85,-0.84,1.71,-0.45,2.22,0.27,-0.75,0.009074,1.700926


In [16]:
table = FF_mom_DEV[['Mkt-RF', 'SMB', 'HML-DEV', 'RMW', 'CMA']]

print_model = pd.DataFrame({"empty": range(0)})

colnames = table.columns

for col in table.columns:
    y = table[col]
    X = table.drop(col, axis = 1)
    X = sm.add_constant(X)
    X.rename(columns={'const': "Intercept Annual" }, inplace = True)
    
    model = sm.OLS(y, X).fit()
    
    t = pd.DataFrame(model.tvalues)
    
    params = pd.DataFrame(model.params)
    params.iloc[0] = params.iloc[0]*12/100
    params.iloc[0] =  params.iloc[0].apply("{:.2%}".format)
    
    r2 = pd.Series(model.rsquared).to_frame(0)
    r2.iloc[0] = r2.iloc[0].apply("{:.2%}".format)
    
    params = params.append(r2)
    
    out_put = params.merge(t, how='left', left_index=True, right_index=True)
    
    out_put.rename(columns={out_put.columns[0]: col,
                            out_put.columns[1]: ""},
                   index={out_put.index[-1]: 'R-Squared'}, inplace=True)
    
    print_model = pd.concat([print_model, out_put], join="outer", axis = 1)

print_model = print_model.iloc[:,1:]
order = colnames.insert(0,'Intercept Annual').insert(len(colnames)+1, 'R-Squared')
print_model.reindex(order).replace(np.nan, "").T

,Intercept Annual,Mkt-RF,SMB,HML-DEV,RMW,CMA,R-Squared
Mkt-RF,9.33%,,0.286894,0.125474,-0.311409,-0.931541,22.55%
,5.06117,,5.36205,1.7096,-4.17202,-8.8521,
SMB,3.03%,0.141001,,0.110874,-0.425373,-0.157704,16.91%
,2.30913,5.36205,,2.15759,-8.43555,-2.03061,
HML-DEV,-1.07%,0.0340092,0.0611463,,0.174096,1.0213,48.63%
,-1.09931,1.7096,2.15759,,4.48882,23.9588,
RMW,4.57%,-0.0799158,-0.222111,0.164834,,-0.289916,16.11%
,4.89236,-4.17202,-8.43555,4.48882,,-5.25368,
CMA,2.98%,-0.11063,-0.0381074,0.447485,-0.134165,,54.09%
,4.68167,-8.8521,-2.03061,23.9588,-5.25368,,


## Table 4

In [17]:
table = FF_mom_DEV[['Mkt-RF', 'SMB', 'HML-DEV', 'RMW', 'CMA', 'UMD']]

print_model = pd.DataFrame({"empty": range(0)})

colnames = table.columns

for col in table.columns:
    y = table[col]
    X = table.drop(col, axis = 1)
    X = sm.add_constant(X)
    X.rename(columns={'const': "Intercept Annual" }, inplace = True)
    
    model = sm.OLS(y, X).fit()
    t = pd.DataFrame(model.tvalues)
    
    params = pd.DataFrame(model.params)
    params.iloc[0] = params.iloc[0]*12/100
    params.iloc[0] =  params.iloc[0].apply("{:.2%}".format)
    
    r2 = pd.Series(model.rsquared).to_frame(0)
    r2.iloc[0] = r2.iloc[0].apply("{:.2%}".format)
    
    params = params.append(r2)
    
    out_put = params.merge(t, how='left', left_index=True, right_index=True)
    
    out_put.rename(columns={out_put.columns[0]: col,
                            out_put.columns[1]: ""},
                   index={out_put.index[-1]: 'R-Squared'}, inplace=True)
    
    print_model = pd.concat([print_model, out_put], join="outer", axis = 1)

print_model = print_model.iloc[:,1:]
order = colnames.insert(0,'Intercept Annual').insert(len(colnames)+1, 'R-Squared')
print_model.reindex(order).replace(np.nan, "").T

,Intercept Annual,Mkt-RF,SMB,HML-DEV,RMW,CMA,UMD,R-Squared
Mkt-RF,10.34%,,0.287071,0.054447,-0.275823,-0.868309,-0.137897,24.11%
,5.60072,,5.41622,0.724582,-3.6998,-8.22206,-3.7407,
SMB,2.85%,0.14386,,0.120926,-0.429342,-0.164155,0.020433,16.99%
,2.13806,5.41622,,2.28109,-8.46828,-2.10105,0.775405,
HML-DEV,0.05%,0.0141488,0.0627065,,0.189472,0.998769,-0.121918,51.74%
,0.0509758,0.724582,2.28109,,5.02702,24.0747,-6.62588,
RMW,4.07%,-0.0714391,-0.2219,0.188845,,-0.303485,0.0520056,17.04%
,4.29549,-3.6998,-8.46828,5.02702,,-5.50414,2.75926,
CMA,2.63%,-0.104001,-0.0392341,0.460343,-0.140343,,0.036486,54.63%
,4.07441,-8.22206,-2.10105,24.0747,-5.50414,,2.84772,


# 1.3.2 25 Portfolios formed on ME and BEME

In [18]:
FF_mom_DEV

,Mkt-RF,SMB,HML,RMW,CMA,RF,UMD,DEV,HML-DEV
Date,,,,,,,,,
196307,-0.39,-0.47,-0.83,0.66,-1.15,0.27,1.00,-0.005923,-0.824077
196308,5.07,-0.79,1.67,0.40,-0.40,0.25,1.03,0.002805,1.667195
196309,-1.57,-0.48,0.18,-0.76,0.24,0.27,0.16,0.002653,0.177347
196310,2.53,-1.29,-0.10,2.75,-2.24,0.29,3.14,-0.021129,-0.078871
196311,-0.85,-0.84,1.71,-0.45,2.22,0.27,-0.75,0.009074,1.700926
...,...,...,...,...,...,...,...,...,...
202005,5.58,1.92,-4.95,0.71,-3.28,0.01,0.67,-0.035662,-4.914338
202006,2.46,1.94,-2.22,0.04,0.34,0.01,-0.76,-0.002976,-2.217024
202007,5.77,-3.03,-1.31,0.55,1.06,0.01,7.61,-0.048735,-1.261265


In [19]:
portfolio_25 = pd.read_csv('25_Portfolios_5x5.csv', header = 9, nrows = 1131)
portfolio_25.rename(columns={portfolio_25.columns[0]: "Date"}, inplace = True)
portfolio_25.set_index('Date', inplace = True)
portfolio_names = portfolio_25.columns

In [20]:
portfolio_25 = FF_mom_DEV.merge(portfolio_25, how = 'left', on = 'Date')

In [21]:
portfolio_25

,Mkt-RF,SMB,HML,RMW,CMA,RF,UMD,DEV,HML-DEV,SMALL LoBM,...,ME4 BM1,ME4 BM2,ME4 BM3,ME4 BM4,ME4 BM5,BIG LoBM,ME5 BM2,ME5 BM3,ME5 BM4,BIG HiBM
Date,,,,,,,,,,,,,,,,,,,,,
196307,-0.39,-0.47,-0.83,0.66,-1.15,0.27,1.00,-0.005923,-0.824077,1.1307,...,-0.8456,-1.7052,-2.0042,-1.6657,-1.8567,0.1401,0.4524,1.2557,-0.4290,-1.1026
196308,5.07,-0.79,1.67,0.40,-0.40,0.25,1.03,0.002805,1.667195,4.2370,...,5.3965,4.7614,6.2068,7.5619,5.3469,5.7710,4.2230,4.7487,8.1419,6.3824
196309,-1.57,-0.48,0.18,-0.76,0.24,0.27,0.16,0.002653,0.177347,-2.8878,...,-2.6893,-2.0139,-1.8941,-3.5823,-1.9943,-1.3721,-0.7720,-0.9852,-0.1905,-3.4963
196310,2.53,-1.29,-0.10,2.75,-2.24,0.29,3.14,-0.021129,-0.078871,1.2885,...,-0.4287,0.9057,2.3459,5.3382,0.6113,5.3261,1.7383,-0.2829,2.4150,0.4857
196311,-0.85,-0.84,1.71,-0.45,2.22,0.27,-0.75,0.009074,1.700926,-3.3751,...,-0.8648,-0.8828,-0.4882,1.1451,3.5388,-1.2561,0.9849,-1.5542,-2.1322,1.3455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202005,5.58,1.92,-4.95,0.71,-3.28,0.01,0.67,-0.035662,-4.914338,13.5510,...,10.6434,8.2561,5.9748,3.3347,1.9301,5.6458,5.5022,3.7983,1.1571,3.7987
202006,2.46,1.94,-2.22,0.04,0.34,0.01,-0.76,-0.002976,-2.217024,11.0265,...,2.1565,0.6645,-0.7638,2.6486,1.8758,5.1105,-1.0308,-0.6403,-1.8727,2.1123
202007,5.77,-3.03,-1.31,0.55,1.06,0.01,7.61,-0.048735,-1.261265,-0.1979,...,5.9581,7.6874,6.3603,2.7130,2.2014,7.7238,5.1651,4.3200,1.7426,2.9417


## Fama and French Five Factors' alphas and the t-statistics

In [22]:
print_alpha = pd.DataFrame({"empty": range(0)})

for portfolios in portfolio_names:
    y = portfolio_25[portfolios]
    X = portfolio_25[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']]
    
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    
    alpha = model.params[0]
    alpha = pd.Series(alpha).to_frame(0)
    t = model.tvalues[0]
    t = pd.Series(t).to_frame(0)
    
    out_put = alpha.merge(t, how='left', left_index=True, right_index=True)
    
    out_put.rename(columns={out_put.columns[0]: "Alpha Monthly",
                            out_put.columns[1]: "t-statistics"},
                   index={out_put.index[0]: portfolios}, inplace=True)
    
    print_alpha = pd.concat([print_alpha, out_put], join="outer", axis = 0)

print_alpha = print_alpha.iloc[:,1:]
print_alpha.sort_values('Alpha Monthly', ascending=False)

,Alpha Monthly,t-statistics
ME4 BM1,0.543950,8.877997
ME1 BM2,0.542480,7.792818
ME1 BM4,0.531965,10.112514
SMALL HiBM,0.502601,8.882228
BIG LoBM,0.487740,11.545203
ME3 BM2,0.422234,7.073246
ME3 BM4,0.405204,7.107620
ME4 BM4,0.399092,5.979384
ME3 BM1,0.394635,6.750923
ME2 BM3,0.384976,7.299306


## GRS test

In [23]:
for portfolios in portfolio_names:
    y = portfolio_25[portfolios]
    X = portfolio_25[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']]
    
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    
    alpha = model.params[0]
    alpha = pd.Series(alpha).to_frame(0)
    t = model.tvalues[0]
    t = pd.Series(t).to_frame(0)
    
    A = np.identity(len(model.params))
    A = A[1:,:]

print(model.f_test(A))
# p value is less than 0.05 so we reject null that all alphas are jointly equal to 0

<F test: F=array([[562.68084833]]), p=5.060723039500467e-239, df_denom=681, df_num=5>


## AQR six-factor model's alphas and the t-statistics

In [31]:
print_alpha = pd.DataFrame({"empty": range(0)})

for portfolios in portfolio_names:
    y = portfolio_25[portfolios]
    X = portfolio_25[['Mkt-RF', 'SMB', 'HML-DEV', 'RMW', 'CMA', 'UMD']]
    
    X = sm.add_constant(X)    
    model = sm.OLS(y, X).fit()
    
    alpha = model.params[0]
    alpha = pd.Series(alpha).to_frame(0)
    
    t = model.tvalues[0]
    t = pd.Series(t).to_frame(0)
    
    out_put = alpha.merge(t, how='left', left_index=True, right_index=True)
    
    out_put.rename(columns={out_put.columns[0]: "Alpha Monthly",
                            out_put.columns[1]: "t-statistics"},
                   index={out_put.index[0]: portfolios}, inplace=True)
    
    print_alpha = pd.concat([print_alpha, out_put], join="outer", axis = 0)

print_alpha = print_alpha.iloc[:,1:]
print_alpha.sort_values('Alpha Monthly', ascending=False)

,Alpha Monthly,t-statistics
ME4 BM1,0.530219,8.525534
ME1 BM2,0.520157,7.374454
SMALL HiBM,0.516382,8.992053
ME1 BM4,0.510783,9.602799
BIG LoBM,0.490052,11.416769
ME3 BM4,0.429120,7.436433
ME3 BM2,0.428146,7.062833
ME4 BM4,0.420623,6.214860
ME2 BM3,0.409508,7.683229
ME3 BM1,0.409376,6.906091


## GRS test

In [32]:
for portfolios in portfolio_names:
    y = portfolio_25[portfolios]
    X = portfolio_25[['Mkt-RF', 'SMB', 'HML-DEV', 'RMW', 'CMA', 'UMD']]
    
    X = sm.add_constant(X)    
    model = sm.OLS(y, X).fit()
    
    A = np.identity(len(model.params))
    A = A[1:,:]

print(model.f_test(A))  
# p value is less than 0.05 so we reject null that all alphas are jointly equal to 0

<F test: F=array([[474.43463299]]), p=3.403256164204729e-239, df_denom=680, df_num=6>


# 1.3.3 10 Industry Portfolios

In [33]:
portfolio_10_industry = pd.read_csv('10_Industry_Portfolios.csv', header = 6, nrows = 1131)
portfolio_10_industry.rename(columns={portfolio_10_industry.columns[0]: "Date"}, inplace = True)
portfolio_10_industry.set_index('Date', inplace = True)
industry_names = portfolio_10_industry.columns

In [34]:
portfolio_10_industry = FF_mom_DEV.merge(portfolio_10_industry, how = 'left', on = 'Date')

In [35]:
portfolio_10_industry

,Mkt-RF,SMB,HML,RMW,CMA,RF,UMD,DEV,HML-DEV,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other
Date,,,,,,,,,,,,,,,,,,,
196307,-0.39,-0.47,-0.83,0.66,-1.15,0.27,1.00,-0.005923,-0.824077,-0.49,-0.22,-1.41,2.29,-0.69,-0.23,-1.03,0.56,0.80,-1.61
196308,5.07,-0.79,1.67,0.40,-0.40,0.25,1.03,0.002805,1.667195,4.89,6.55,6.20,3.93,5.14,4.29,6.43,9.56,4.20,5.49
196309,-1.57,-0.48,0.18,-0.76,0.24,0.27,0.16,0.002653,0.177347,-1.69,-0.24,-0.76,-3.64,0.13,2.36,0.97,-4.06,-2.50,-3.16
196310,2.53,-1.29,-0.10,2.75,-2.24,0.29,3.14,-0.021129,-0.078871,2.65,9.72,2.58,-0.32,8.29,3.40,0.52,3.38,-0.67,1.38
196311,-0.85,-0.84,1.71,-0.45,2.22,0.27,-0.75,0.009074,1.700926,-1.13,-4.84,0.30,-1.15,-0.29,4.16,-1.23,-1.65,-1.02,0.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202005,5.58,1.92,-4.95,0.71,-3.28,0.01,0.67,-0.035662,-4.914338,3.30,7.30,5.96,0.55,8.33,4.86,4.38,3.97,4.57,4.05
202006,2.46,1.94,-2.22,0.04,0.34,0.01,-0.76,-0.002976,-2.217024,-0.03,14.20,2.56,-0.36,6.00,-2.44,4.21,-1.76,-5.04,0.00
202007,5.77,-3.03,-1.31,0.55,1.06,0.01,7.61,-0.048735,-1.261265,5.85,18.73,4.48,-4.77,6.90,5.09,9.53,4.44,6.37,3.62


## Fama and French Five Factors' alphas and the t-statistics

In [50]:
print_alpha = pd.DataFrame({"empty": range(0)})

for portfolios in industry_names:
    y = portfolio_10_industry[portfolios]
    X = portfolio_10_industry[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']]
    
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    
    alpha = model.params[0]
    alpha = pd.Series(alpha).to_frame(0)
    t = model.tvalues[0]
    t = pd.Series(t).to_frame(0)
    
    out_put = alpha.merge(t, how='left', left_index=True, right_index=True)
    
    out_put.rename(columns={out_put.columns[0]: "Alpha Monthly",
                            out_put.columns[1]: "t-statistics"},
                   index={out_put.index[0]: portfolios}, inplace=True)
    
    print_alpha = pd.concat([print_alpha, out_put], join="outer", axis = 0)

print_alpha = print_alpha.iloc[:,1:]
print_alpha.sort_values('Alpha Monthly', ascending=False)

,Alpha Monthly,t-statistics
HiTec,0.803585,7.791913
Hlth,0.561988,4.891174
Telcm,0.462552,3.962789
Shops,0.351026,3.666637
Utils,0.329828,2.707726
NoDur,0.307979,3.754408
Manuf,0.211559,3.438245
Other,0.186758,2.887648
Enrgy,0.143255,0.859130
Durbl,0.016038,0.109366


## GRS Test

In [51]:
for portfolios in industry_names:
    y = portfolio_10_industry[portfolios]
    X = portfolio_10_industry[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']]
    
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    
    alpha = model.params[0]
    alpha = pd.Series(alpha).to_frame(0)
    t = model.tvalues[0]
    t = pd.Series(t).to_frame(0)
    
    B = np.identity(len(model.params))
    B = B[1:,:]

print(model.f_test(B))
# p value is less than 0.05 so we reject null that all alphas are jointly equal to 0

<F test: F=array([[1309.40516429]]), p=0.0, df_denom=681, df_num=5>


## AQR six-factor model's alphas and the t-statistics

In [46]:
print_alpha = pd.DataFrame({"empty": range(0)})

for portfolios in industry_names:
    y = portfolio_10_industry[portfolios]
    X = portfolio_10_industry[['Mkt-RF', 'SMB', 'HML-DEV', 'RMW', 'CMA', 'UMD']]
    
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    
    alpha = model.params[0]
    alpha = pd.Series(alpha).to_frame(0)
    t = model.tvalues[0]
    t = pd.Series(t).to_frame(0)
    
    out_put = alpha.merge(t, how='left', left_index=True, right_index=True)
    
    out_put.rename(columns={out_put.columns[0]: "Alpha Monthly",
                            out_put.columns[1]: "t-statistics"},
                   index={out_put.index[0]: portfolios}, inplace=True)
    
    print_alpha = pd.concat([print_alpha, out_put], join="outer", axis = 0)

print_alpha = print_alpha.iloc[:,1:]
print_alpha.sort_values('Alpha Monthly', ascending=False)

,Alpha Monthly,t-statistics
HiTec,0.839686,8.040569
Hlth,0.525709,4.513634
Telcm,0.516715,4.380918
Shops,0.409110,4.245658
NoDur,0.322318,3.871246
Utils,0.295620,2.393656
Manuf,0.246432,3.973952
Other,0.217500,3.328736
Durbl,0.202697,1.417057
Enrgy,0.107320,0.634310


## GRS Test

In [53]:
for portfolios in industry_names:
    y = portfolio_10_industry[portfolios]
    X = portfolio_10_industry[['Mkt-RF', 'SMB', 'HML-DEV', 'RMW', 'CMA', 'UMD']]
    
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    
    B = np.identity(len(model.params))
    B = B[1:,:]

print(model.f_test(B)) 
# p value is less than 0.05 so we reject null that all alphas are jointly equal to 0

<F test: F=array([[1102.5271599]]), p=0.0, df_denom=680, df_num=6>
